<a href="https://colab.research.google.com/github/PedroAdair/NLP_proyectos/blob/main/Tweets_Politicos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelos de lenguaje para tuits de politicos mexicanos
##Pedro Adair Gallegos Avila

Nuestro método de escritura es una manera de caracterizarnos como individuos, y en este ejercicio tratamos de entrenar un modelo que a nivel de palabras tenga la habilidad de escribir mensajes tal cual lo haría cada uno de los siguientes políticos mexicanos:


1.   Claudia Sheinbaum
2.   Ricardo Anaya
3.   Andres Manuel
4.   Samuel Garcia
5.   Marcelo Ebrard

El primer paso consiste en leer y preprocesar nuestros tuits, para ello haremos uso de las herramientas de la librería NLTK, entre las cuales destacan la eliminación de mayúsculas, quitar emojis (algo importante y que redujo el modelo de Samuel García) entre otros.
Algo que note ya cuando tenía algunos modelos entrenados es que entre mayores longitudes les dé al modelo, cuando intento generar texto, al final aparece la sentencia "http", puesto que, en el corpus, al final de cada tuit aparece la liga que lleva al comentario original. En una siguiente fase consideraría eliminar este token del preprocesado.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Instalación y carga de librerias

In [ ]:
#@title instalación
!pip install datasets

In [ ]:
#@title Librerias
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

import torch
import torch.nn as nn
import torch.optim as optim
import math
import torchtext
import datasets
from torch.utils.data import DataLoader
from tqdm import tqdm


from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

import re, string, unicodedata
import nltk

import inflect #convertir números en palabras

from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer



In [ ]:
#@title Funciones NLTK
"""
NLKT es la opción que selecciones para preprocesar mis textos
"""
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from nltk import WordPunctTokenizer

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

#Para stop words en español
es_stop = set(nltk.corpus.stopwords.words('spanish'))

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

## Carga y preprocesamiento

El primer paso consiste en cargar los tweets, preprocesarlos, así como darles  el formato dataset de HugginFace

In [ ]:
#@title Los Twets de los politicos
ClaudiaShen   = "/content/drive/MyDrive/Tarea4_TSCD/tweets_politicos/@Claudiashein_tweets.csv"
RicardoAnaya  = "/content/drive/MyDrive/Tarea4_TSCD/tweets_politicos/@RicardoAnayaC_tweets.csv"
AndresManuel  = "/content/drive/MyDrive/Tarea4_TSCD/tweets_politicos/@lopezobrador__tweets.csv"
MarceloEbrard = "/content/drive/MyDrive/Tarea4_TSCD/tweets_politicos/@m_ebrard_tweets.csv"
SamuelGarcia  = "/content/drive/MyDrive/Tarea4_TSCD/tweets_politicos/@samuel_garcias_tweets.csv"

In [ ]:
#@title Ejemplo con Claudia Sheinbaum
df = pd.read_csv(ClaudiaShen)
twets = df["text"]
print(twets[0])

Este fin de semana 2 mil representantes de todo el país se dieron cita en la Ciudad de México para juntos construir territorios de derechos, paz y bienestar. 

#EncuentroPorElBienestar https://t.co/F840VPVrX7


###Preprocesamiento
Cuando un usuario de Tweeter escribe un mensaje, puede usar multiples simbolos de lenguaje, tal es es caso de emojis, añadir hiperlinks, etc. Todos estos factores complican la tarea de generar un modelo de generación de texto. Para este caso vamos a eliminar estos caracteres especiales, ademas de homogeneizar el texto, al pasar todo a minusculas, eliminar espacios multiples, quitar caracteres especiales, entre otros.

In [ ]:
#@title Preprocesar un tweet con la función     preprocess_text()

stemmer = WordNetLemmatizer()
word_punctuation_tokenizer = nltk.WordPunctTokenizer()

def preprocess_text(document):
        # Remove all the special characters
        document = re.sub(r'\W', ' ', str(document))

        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

        # Remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)

        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)

        # Converting to Lowercase
        document = document.lower()

        # Lemmatization
        tokens = document.split()
        tokens = [stemmer.lemmatize(word) for word in tokens]
        tokens = [word for word in tokens if word not in es_stop]
        tokens = [word for word in tokens if len(word) > 2]

        preprocessed_text = ' '.join(tokens)

        return preprocessed_text

### Partición de conjuntos
Para el entrenamiento del modelo, creamos una partición de conjuntos de entrenamiento/validación/prueba de la siguiente forma

*   Train: 75%
*   Valid: 15%
*   Test:  10%

Usamos la semilla 42 para replicabilidad de los resultados.

La función **leer_twets()** recibe como entrada el nombre del politico y realiza  el preprocesamiento del texto, ademas de generar la partición de entrenamiento/validación/test en el formato de dataset de Huggingface

In [ ]:
#@title Construcción de leer_twets()
def leer_twets(nombre_politico):
  #leemos el archivo con los twets correspondientes
  df = pd.read_csv(nombre_politico)
  #Vamos a generar una partición (el id no lo usamos, simplemente es para llenar el requisito de train_test_split)
  twets = df["text"]
  twets = twets.map(preprocess_text)
  id_twets = df["id"]

  X_train, X_val_test, y_train, y_val_test = train_test_split(twets, id_twets, test_size=0.25, random_state=42)
  X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.40, random_state=42)

  #Una vez generamos los conjuntos, debemos de guardarlos (temporalmente) como csv par luego
  #volverlos del tipo dataset de Hu
  X_train = X_train.to_frame()
  X_val = X_val.to_frame()
  X_test = X_test.to_frame()

  train_dataset = Dataset.from_pandas(X_train)
  valid_dataset = Dataset.from_pandas(X_val)
  test_dataset  = Dataset.from_pandas(X_test)

  return(train_dataset, valid_dataset, test_dataset)

El ejemplo de Claudia Sheinbaum

In [ ]:
train_dataset, valid_dataset, test_dataset = leer_twets(ClaudiaShen)

In [ ]:
train_dataset['text'][10]

'cumplimos brindamos derecho vivienda familias heriberto frías 1301 comparto imágenes después http uztkri6rvz'

## Tokenización y construcción del vocabulario
El siguiente paso consiste en la creación de nuestro vocabulario, para lo cual debemos de tokenizar nuestros documentos

In [ ]:
#@title tokenize_data()
def tokenize_data(example,tokenizer):
  return  {'tokens': tokenizer(example['text'])}

Ahora creamos la función que tokeniza nuestros datasets

In [ ]:
#@title Se crean los tokens de los datasets
tokenizer = get_tokenizer('basic_english')
def TokenizerDatasets(train_dataset,valid_dataset,test_dataset):
  tokenizer = get_tokenizer('basic_english')
  tokenized_train_dataset = train_dataset.map(tokenize_data, remove_columns=['text'], fn_kwargs={'tokenizer': tokenizer})
  tokenized_valid_dataset = valid_dataset.map(tokenize_data, remove_columns=['text'], fn_kwargs={'tokenizer': tokenizer})
  tokenized_test_dataset = test_dataset.map(tokenize_data, remove_columns=['text'], fn_kwargs={'tokenizer': tokenizer})

  return(tokenized_train_dataset, tokenized_valid_dataset, tokenized_test_dataset)

In [ ]:
tokenized_train_dataset, tokenized_valid_dataset, tokenized_test_dataset = TokenizerDatasets(train_dataset,valid_dataset,test_dataset)

  0%|          | 0/2437 [00:00<?, ?ex/s]

  0%|          | 0/487 [00:00<?, ?ex/s]

  0%|          | 0/326 [00:00<?, ?ex/s]

Como tenemos muy pocos twets, lo que haremos sera construir nuestro vocabulario con tokens de los datasets de entrenamiento y validación

In [ ]:
#@title Construcción del vocabulario
def build_vocab(dataset):
    tokens = dataset.map(tokenize_data, remove_columns=['text'], fn_kwargs={'tokenizer': tokenizer})
    return tokens


In [ ]:
vocab = build_vocab_from_iterator(build_vocab(datasets.concatenate_datasets([train_dataset, valid_dataset, test_dataset]))['tokens'], specials=["<UNK>", "<EOS>"], min_freq=3)
vocab.set_default_index(vocab["<UNK>"])

  0%|          | 0/3250 [00:00<?, ?ex/s]

In [ ]:
#print(len(vocab))
print(vocab.get_itos()[:10])

['<UNK>', '<EOS>', 'http', 'ciudad', 'hoy', 'méxico', 'gracias', 'día', 'envivo', 'años']


A continuación un ejemplo de una frase y como se representa en nuestro diccionario, notemos que la palabra "renuncia" no aparece en el diccionario, por lo cual te corresponde el token <UKN> con valor 0.

In [ ]:
tokens = tokenizer("renuncia director  imss   Xochimilco tras visita del presidente")
indexes = vocab(tokens)

tokens, indexes

(['renuncia',
  'director',
  'imss',
  'xochimilco',
  'tras',
  'visita',
  'del',
  'presidente'],
 [0, 532, 2194, 151, 820, 194, 0, 12])

## Dataloader y construcción del modelo de lenguaje

Para nuestro modelo, las secuencias deben de ser procesadas en secuencias de una longitud definida.

In [ ]:
#@title batchify
def batchify(dataset, vocab, batch_size):
    data = []
    for example in dataset:
        if example['tokens']:
            tokens = example['tokens'].append('<EOS>')
            tokens = [vocab[token] for token in example['tokens']]
            data.extend(tokens)
    data = torch.LongTensor(data)
    num_batches = data.shape[0] // batch_size
    data = data[:num_batches * batch_size]
    data = data.view(batch_size, num_batches)
    return data

Como nuestro corpus es pequeño, definimos un batch size de tamaño 32

In [ ]:
batch_size = 32
train_data = batchify(tokenized_train_dataset, vocab, batch_size)
valid_data = batchify(tokenized_valid_dataset, vocab, batch_size)
test_data = batchify(tokenized_test_dataset, vocab, batch_size)

In [ ]:
print("train : ",train_data.shape)
print("test  : ",test_data.shape)

train :  torch.Size([32, 1341])
test  :  torch.Size([32, 187])


##Construcción de modelo
En este caso, se adopta un modelo LSTM con las siguientes caracteristicas:


---



* El vocabulario tendra todas sus palabras, por lo que no se quitara ninguna
* La dimension del embeddig se queda en 500 elementos
* La dimension de la capa oculta se fija en 1200
* Se consideraran 3 capas
* Un dropout por default de 0.5
* Un learning rate muy chico 0.001
* Función de perdida: CrossEntropyLoss
* Optimizador: Adam
* Longitud de secuencia *seq_len= 40*
* Clip:  0.25

In [ ]:
#@title Modelo LSTM
class LSTM_LM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers,
                    dropout=dropout_rate, batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_dim, vocab_size)

        self.init_weights()

    def forward(self, src, hidden):
        embedding = self.dropout(self.embedding(src))
        output, hidden = self.lstm(embedding, hidden)
        output = self.dropout(output)
        prediction = self.fc(output)
        return prediction, hidden

    def init_weights(self):
        init_range_emb = 0.1
        init_range_other = 1/math.sqrt(self.hidden_dim)
        self.embedding.weight.data.uniform_(-init_range_emb, init_range_emb)
        self.fc.weight.data.uniform_(-init_range_other, init_range_other)
        self.fc.bias.data.zero_()
        for i in range(self.num_layers):
            self.lstm.all_weights[i][0] = torch.FloatTensor(self.embedding_dim,
                    self.hidden_dim).uniform_(-init_range_other, init_range_other)
            self.lstm.all_weights[i][1] = torch.FloatTensor(self.hidden_dim,
                    self.hidden_dim).uniform_(-init_range_other, init_range_other)

    def init_hidden(self, batch_size, device):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device)
        return hidden, cell

    def detach_hidden(self, hidden):
        hidden, cell = hidden
        hidden = hidden.detach()
        cell = cell.detach()
        return hidden, cell

Tenemos los siguientes hiperparametros:

In [ ]:
vocab_size = len(vocab)
embedding_dim = 500
hidden_dim = 1200
num_layers = 2
dropout_rate = 0.5
lr = 1e-3
n_epochs = 100
seq_len = 40
clip = 0.2

Preparamos el modelo, así como visualizar la cantidad de parametros que vamos a aprender

In [ ]:
model = LSTM_LM(vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {num_params:,} trainable parameters')

The model has 25,082,865 trainable parameters


In [ ]:
def get_batch(data, seq_len, num_batches, idx): #(data, seq_len, num_batches, idx)
    src = data[:, idx:idx+seq_len]
    target = data[:, idx+1:idx+seq_len+1]
    return src, target

##Entrenamiento y Evaluación del modelo
A continuación definimos las funciones con las que vamos a entrenar nuestro modelo, así como algunas auxiliares

In [ ]:
#@title Obtención de los batches
def get_batch(data, seq_len, idx):
    src = data[:, idx:idx+seq_len]
    target = data[:, idx+1:idx+seq_len+1]
    return src, target

In [ ]:
#@title entrenamiento del modelo
def train(model, data, optimizer, criterion, batch_size, seq_len, clip, device):
    epoch_loss = 0
    model.train()
    # drop all batches that are not a multiple of seq_len
    num_batches = data.shape[-1]
    data = data[:, :num_batches - (num_batches -1) % seq_len]
    num_batches = data.shape[-1]

    hidden = model.init_hidden(batch_size, device)
    # The last batch can't be a src
    for idx in tqdm(range(0, num_batches - 1, seq_len), desc='Training: ',leave=False):
        #zero the gradients due to the previous batch and detach its hidden state
        optimizer.zero_grad()
        hidden = model.detach_hidden(hidden)

        src, target = get_batch(data, seq_len, idx)
        src, target = src.to(device), target.to(device)
        batch_size = src.shape[0]
        prediction, hidden = model(src, hidden)

        prediction = prediction.reshape(batch_size * seq_len, -1)
        target = target.reshape(-1)
        loss = criterion(prediction, target)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item() * seq_len
    return epoch_loss / num_batches

In [ ]:
#@title evaluación del modelo
def evaluate(model, data, criterion, batch_size, seq_len, device):
    epoch_loss = 0
    model.eval()
    num_batches = data.shape[-1]
    data = data[:, :num_batches - (num_batches -1) % seq_len]
    num_batches = data.shape[-1]

    hidden = model.init_hidden(batch_size, device)

    with torch.no_grad():
        for idx in range(0, num_batches - 1, seq_len):
            hidden = model.detach_hidden(hidden)
            src, target = get_batch(data, seq_len,  idx)
            src, target = src.to(device), target.to(device)
            batch_size= src.shape[0]

            prediction, hidden = model(src, hidden)
            prediction = prediction.reshape(batch_size * seq_len, -1)
            target = target.reshape(-1)

            loss = criterion(prediction, target)
            epoch_loss += loss.item() * seq_len
    return epoch_loss / num_batches

Para guardar el modelo, tenemos el valor *saved*, cuando se toma como false, se entrena y guarda el modelo, mientras qeu si es True, se lee el modelo previamente guardado

In [ ]:
saved = True

#reduce the learning rate by a factor of 2 after every epoch associated with no improvement
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=0)

Como ya he entrenado los modelos, tengo los saved=True

In [ ]:
#@title entrenamiento del modelo de prueba
if saved:
    model.load_state_dict(torch.load('/content/drive/MyDrive/Tarea4_TSCD/tweets_politicos/best-val-ClaudiaShein.pt',  map_location=device))
    test_loss = evaluate(model, test_data, criterion, batch_size, seq_len, device)
    print(f'Test Perplexity: {math.exp(test_loss):.3f}')
else:
    best_valid_loss = float('inf')

    for epoch in range(n_epochs):
        train_loss = train(model, train_data, optimizer, criterion,
                    batch_size, seq_len, clip, device)
        valid_loss = evaluate(model, valid_data, criterion, batch_size,
                    seq_len, device)

        lr_scheduler.step(valid_loss)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), '/content/drive/MyDrive/Tarea4_TSCD/tweets_politicos/best-val-ClaudiaShein.pt')
        print(f'Epoch: {epoch}')
        print(f'\tTrain Perplexity: {math.exp(train_loss):.3f}')
        print(f'\tValid Perplexity: {math.exp(valid_loss):.3f}')

Test Perplexity: 288.153


## Inferencia

Finalmente, la inferencia la realizaremos con la función generate.

In [ ]:
max_seq_len=10
seed = 2022
max_seq_len = 8
#@title Etapa de inferencia
def generate(prompt, max_seq_len, model, tokenizer, vocab, device, seed=None):
    if seed is not None:
        torch.manual_seed(seed)
    model.eval()
    tokens = tokenizer(prompt)
    indices = [vocab[t] for t in tokens]
    batch_size = 1
    hidden = model.init_hidden(batch_size, device)
    with torch.no_grad():
        for i in range(max_seq_len):
            src = torch.LongTensor([indices]).to(device)
            prediction, hidden = model(src, hidden)
            probs = torch.softmax(prediction[:, -1], dim=-1)
            prediction = torch.multinomial(probs, num_samples=1).item()

            while prediction == vocab['<UNK>']:
                prediction = torch.multinomial(probs, num_samples=1).item()

            if prediction == vocab['<EOS>']:
                break

            indices.append(prediction)

    itos = vocab.get_itos()
    tokens = [itos[i] for i in indices]
    return tokens

#5 politicos

Una vez he explicado la metodología que adopte para resolver este problema, es que entreno un modelo de lenguaje para cada uno de los políticos. A continuación, algunos resultados/interpretaciones del modelo.

**Nota:** Para el apartado b del ejercicio, donde realizamos la "destilación", la aplicamos únicamente para Samuel Garcia y Marcelo Ebrard. Elegí 5 palabras para cada político, algunas se repiten para más de 1, mientras que otras  algo que esperaba escuchar de la persona en cuestión.


## Claudia Sheinbaum [Twitter](https://twitter.com/Claudiashein?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Eauthor)

Se entreno el modelo para generación de texto como lo haria la jefa de gobierno de la Ciudad de México. Para la etapa de inferencia, se dieron las siguientes palabras:


1.    seguridad
2.    gobierno
3.    salud
4.    infraestructura
5.    economía

Algo que no habia notado, es el hecho de que todos los twetts al final tienen la liga al comentario original, y estos modelos de lenguaje tratan de realizar esta acción con el token "http". Fuera de este inconveniente, me parece consisitente a la forma de escribir de la jefa de gobierno, ya que trata de enmarcar las buenas acciones de mis palabras iniciales, por ejemplo:


1.    seguridad / captación  
2.    gobierno / resultados  
3.    salud / vacunación
4.    infraestructura / vialidades
5.    economía / habitantes

Los pesos de este modelo estan guardados en la siguiente [carpeta](/content/drive/MyDrive/Tarea4_TSCD/tweets_politicos), el archivo es **best-val-ClaudiaShein.pt**

### Resultados

In [ ]:
prompt = 'seguridad'

generation = generate(prompt, max_seq_len, model, tokenizer, vocab, device, seed)
print('Generated text:\n'+' '.join(generation)+'\n')

Generated text:
seguridad captación segura http ciudad http años



In [ ]:
prompt = 'gobierno'
generation = generate(prompt, max_seq_len, model, tokenizer, vocab, device)
print('Generated text:\n'+' '.join(generation)+'\n')

Generated text:
gobierno únicamente fuerte resultados ambiente sembrandoparques http bienestar http



In [ ]:
prompt = 'salud'
generation = generate(prompt, max_seq_len, model, tokenizer, vocab, device)
print('Generated text:\n'+' '.join(generation)+'\n')

Generated text:
salud coordinados vacunación toda licitación sedes http http



In [ ]:
prompt = 'infraestructura'
generation = generate(prompt, max_seq_len, model, tokenizer, vocab, device)
print('Generated text:\n'+' '.join(generation)+'\n')

Generated text:
infraestructura familias vialidades nueva cómodo ideales cultural felizjueves http



In [ ]:
prompt = 'economía'
generation = generate(prompt, max_seq_len, model, tokenizer, vocab, device)
print('Generated text:\n'+' '.join(generation)+'\n')

Generated text:
economía ponte acercar habitantes tod http vida http día



En este ultimo ejemplo, comenze a ver que se repetian palabras despues de cierta longitud.

## Ricardo Anaya [Twitter](https://twitter.com/RicardoAnayaC?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Eauthor)

El siguiente político que analice fue acerca del excandidato presidencial, que se ha marcado como una fuerza de oposición, algo que naturalmente espero de su modelo de lenguaje, seria remarcar las acciones negativas del gobierno, tendencias diferentes a las del actual mandatario, etc. Para este personaje se evaluar en las siguientes palabras:


1.   seguridad    
2.   gobierno
3.   salud
4.   economía
5.   justicia

Algo que me llamo la atención, es que su modelo de lenguaje no parece hablar sobre la oposición, sino sobre sus acciones y las de su partido.

1.   seguridad / calderón
2.   gobierno / movciudadanomx  
3.   salud /  ganóelpan
4.   economía / pobres
5.   justicia / acciónnacional



De forma similar a los twits de Claudia, los preprocesamos, tokenizamos, construimos el diccionario (incluyendo tokens especiales, entrenamos el modelo) y guardamos los pesos en la misma carpeta, salvo que estos con el nombre **best-val-RicardoAnaya.pt**

### Construcción del modelo de lenguaje

In [ ]:
train_dataset, valid_dataset, test_dataset = leer_twets(RicardoAnaya)
tokenized_train_dataset, tokenized_valid_dataset, tokenized_test_dataset = TokenizerDatasets(train_dataset,valid_dataset,test_dataset)

  0%|          | 0/2416 [00:00<?, ?ex/s]

  0%|          | 0/483 [00:00<?, ?ex/s]

  0%|          | 0/323 [00:00<?, ?ex/s]

In [ ]:
vocab = build_vocab_from_iterator(build_vocab(datasets.concatenate_datasets([train_dataset, valid_dataset, test_dataset]))['tokens'], specials=["<UNK>", "<EOS>"], min_freq=3)
vocab.set_default_index(vocab["<UNK>"])

  0%|          | 0/3222 [00:00<?, ?ex/s]

In [ ]:
#@title solo para ver que no nos hemos atrasado con el modelo
print(vocab.get_itos()[:10])


['<UNK>', '<EOS>', 'http', 'méxico', 'vamos', 'cambio', 'gobierno', 'hoy', 'pan', 'accionnacional']


In [ ]:
batch_size = 32
train_data = batchify(tokenized_train_dataset, vocab, batch_size)
valid_data = batchify(tokenized_valid_dataset, vocab, batch_size)
test_data = batchify(tokenized_test_dataset, vocab, batch_size)

In [ ]:
vocab_size = len(vocab)
embedding_dim = 500
hidden_dim = 1200
num_layers = 2
dropout_rate = 0.5
lr = 1e-3
n_epochs = 100
seq_len = 40
clip = 0.25

In [ ]:
model = LSTM_LM(vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {num_params:,} trainable parameters')

The model has 23,830,929 trainable parameters


In [ ]:
saved = True

#reduce the learning rate by a factor of 2 after every epoch associated with no improvement
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=0)

In [ ]:
if saved:
    model.load_state_dict(torch.load('/content/drive/MyDrive/Tarea4_TSCD/tweets_politicos/best-val-RicardoAnaya.pt',  map_location=device))
    test_loss = evaluate(model, test_data, criterion, batch_size, seq_len, device)
    print(f'Test Perplexity: {math.exp(test_loss):.3f}')
else:
    best_valid_loss = float('inf')

    for epoch in range(n_epochs):
        train_loss = train(model, train_data, optimizer, criterion,
                    batch_size, seq_len, clip, device)
        valid_loss = evaluate(model, valid_data, criterion, batch_size,
                    seq_len, device)

        lr_scheduler.step(valid_loss)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), '/content/drive/MyDrive/Tarea4_TSCD/tweets_politicos/best-val-RicardoAnaya.pt')
        print(f'Epoch: {epoch}')
        print(f'\tTrain Perplexity: {math.exp(train_loss):.3f}')
        print(f'\tValid Perplexity: {math.exp(valid_loss):.3f}')

Test Perplexity: 245.089


Train Perplexity: 115.940

Valid Perplexity: 254.115


###Resultados

In [ ]:
prompt = 'seguridad'

generation = generate(prompt, max_seq_len, model, tokenizer, vocab, device, seed)
print('Generated text:\n'+' '.join(generation)+'\n')

Generated text:
seguridad calderón escuchar síguela frente paz país seguiremos trabajando



In [ ]:
prompt = 'gobierno'
generation = generate(prompt, max_seq_len, model, tokenizer, vocab, device)
print('Generated text:\n'+' '.join(generation)+'\n')

Generated text:
gobierno distractores movciudadanomx sepuede http



In [ ]:
prompt = 'salud'
generation = generate(prompt, max_seq_len, model, tokenizer, vocab, device)
print('Generated text:\n'+' '.join(generation)+'\n')

Generated text:
salud adelante combate ganóelpan http



In [ ]:
prompt = 'economía'
generation = generate(prompt, max_seq_len, model, tokenizer, vocab, device)
print('Generated text:\n'+' '.join(generation)+'\n')

Generated text:
economía transparente pobres pueblo apoyar aumentado porlamañana http



In [ ]:
prompt = 'justicia'
generation = generate(prompt, max_seq_len, model, tokenizer, vocab, device)
print('Generated text:\n'+' '.join(generation)+'\n')

Generated text:
justicia acciónnacional mando privada junto defrentealfuturo http ciudadanos



## Andres Manuel [Twitter](https://twitter.com/lopezobrador_?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Eauthor)

Cuando evalúe el modelo de lenguaje asociado al presidente de la república, pude notar que, a diferencia de Claudia Sheinbaum, al introducir palabras como justicia, economía, seguridad, etc. no recibe como salida un "discurso" donde se intente enmarcar las buenas acciones, sino el nombre de estados/ciudades de México, lo cual es consistente con sus múltiples giras que el mandatario realiza a lo largo de nuestro país. Por otra parte,  ante palabras de enfoques sociales/culturales como medio ambiente o jóvenes, aparecieron palabras asociadas tanto a fenómenos positivos como negativos.

 Finalmente, ante conferencia, evidentemente apareció mañanera.


###Construcción del modelo de lenguaje

In [ ]:
train_dataset, valid_dataset, test_dataset = leer_twets(AndresManuel)
tokenized_train_dataset, tokenized_valid_dataset, tokenized_test_dataset = TokenizerDatasets(train_dataset,valid_dataset,test_dataset)

  0%|          | 0/2437 [00:00<?, ?ex/s]

  0%|          | 0/487 [00:00<?, ?ex/s]

  0%|          | 0/326 [00:00<?, ?ex/s]

In [ ]:
vocab = build_vocab_from_iterator(build_vocab(datasets.concatenate_datasets([train_dataset, valid_dataset, test_dataset]))['tokens'], specials=["<UNK>", "<EOS>"], min_freq=3)
vocab.set_default_index(vocab["<UNK>"])

  0%|          | 0/3250 [00:00<?, ?ex/s]

In [ ]:
#@title solo para ver que no nos hemos atrasado con el modelo
print(vocab.get_itos()[:10])


['<UNK>', '<EOS>', 'http', 'conferencia', 'matutina', 'méxico', 'prensa', 'nacional', 'pueblo', 'palacio']


In [ ]:
batch_size = 32
train_data = batchify(tokenized_train_dataset, vocab, batch_size)
valid_data = batchify(tokenized_valid_dataset, vocab, batch_size)
test_data = batchify(tokenized_test_dataset, vocab, batch_size)

In [ ]:
vocab_size = len(vocab)
embedding_dim = 500
hidden_dim = 1200
num_layers = 2
dropout_rate = 0.5
lr = 1e-3
n_epochs = 100
seq_len = 40
clip = 0.25

In [ ]:
model = LSTM_LM(vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {num_params:,} trainable parameters')

The model has 24,201,747 trainable parameters


In [ ]:
saved = True

#reduce the learning rate by a factor of 2 after every epoch associated with no improvement
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=0)

In [ ]:
if saved:
    model.load_state_dict(torch.load('/content/drive/MyDrive/Tarea4_TSCD/tweets_politicos/best-val-AndresManuel.pt',  map_location=device))
    test_loss = evaluate(model, test_data, criterion, batch_size, seq_len, device)
    print(f'Test Perplexity: {math.exp(test_loss):.3f}')
else:
    best_valid_loss = float('inf')

    for epoch in range(n_epochs):
        train_loss = train(model, train_data, optimizer, criterion,
                    batch_size, seq_len, clip, device)
        valid_loss = evaluate(model, valid_data, criterion, batch_size,
                    seq_len, device)

        lr_scheduler.step(valid_loss)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), '/content/drive/MyDrive/Tarea4_TSCD/tweets_politicos/best-val-AndresManuel.pt')
        print(f'Epoch: {epoch}')
        print(f'\tTrain Perplexity: {math.exp(train_loss):.3f}')
        print(f'\tValid Perplexity: {math.exp(valid_loss):.3f}')

Test Perplexity: 139.905


Train Perplexity: 56.367

Valid Perplexity: 124.483

###Resultados

In [ ]:
prompt = 'medio ambiente'
generation = generate(prompt, max_seq_len, model, tokenizer, vocab, device)
print('Generated text:\n'+' '.join(generation)+'\n')

Generated text:
medio ambiente ley pobres http nacional



In [ ]:
prompt = 'corrupción'
generation = generate(prompt, max_seq_len, model, tokenizer, vocab, device)
print('Generated text:\n'+' '.join(generation)+'\n')

Generated text:
corrupción pinos conferencia matutina http chiapas



In [ ]:
prompt = 'jóvenes'
generation = generate(prompt, max_seq_len, model, tokenizer, vocab, device)
print('Generated text:\n'+' '.join(generation)+'\n')

Generated text:
jóvenes unam años millones social mina 2021 http pcr7jyirjl



In [ ]:
prompt = 'Conferencia'
generation = generate(prompt, max_seq_len, model, tokenizer, vocab, device)
print('Generated text:\n'+' '.join(generation)+'\n')

Generated text:
conferencia matutina vivo http pcr7jyirjl



#Destilado
Un denomeno que ocurrio para todos los modelos, fue la repetición de palabras a partir de cierta longitud, y en algunos casos, esta longitud fue muy corta. Una alternativa sencilla (aunque con fundamento teorico bastante profundo) consiste en  modificar la distribución de probabilidades con la que softmax construye las salidas. Para esto es que tenemos la función   **generate_destilada()**, que recibe como parametro adicional la temperatura (default 20). A continuacón algunos ejemplos con los 2 politicos faltantes.

In [ ]:

#@title generate_destilada
def generate_destilada(prompt, max_seq_len, model, tokenizer, vocab, device, seed=None, temp=20):
    if seed is not None:
        torch.manual_seed(seed)
    model.eval()
    tokens = tokenizer(prompt)
    indices = [vocab[t] for t in tokens]
    batch_size = 1
    hidden = model.init_hidden(batch_size, device)
    with torch.no_grad():
        for i in range(max_seq_len):
            src = torch.LongTensor([indices]).to(device)
            prediction, hidden = model(src, hidden)
            #Aqui aparece la modificacion, donde dividimos el vector que entra al
            #softmax por este parametro de temperatura.
            probs = torch.softmax(prediction[:, -1]/temp, dim=-1)
            prediction = torch.multinomial(probs, num_samples=1).item()

            while prediction == vocab['<UNK>']:
                prediction = torch.multinomial(probs, num_samples=1).item()

            if prediction == vocab['<EOS>']:
                break

            indices.append(prediction)

    itos = vocab.get_itos()
    tokens = [itos[i] for i in indices]
    return tokens

## Samuel García [Twitter](https://twitter.com/samuel_garcias?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Eauthor)

Al ser el político más joven de la lista, pensaba que sería el más difícil de generar lenguaje similar al suyo, puesto que en sus tuits hace uso de muchos emojis que fueron eliminados en su preprocesado.

###Construcción del modelo de lenguaje

In [ ]:
train_dataset, valid_dataset, test_dataset = leer_twets(SamuelGarcia)
tokenized_train_dataset, tokenized_valid_dataset, tokenized_test_dataset = TokenizerDatasets(train_dataset,valid_dataset,test_dataset)

  0%|          | 0/2433 [00:00<?, ?ex/s]

  0%|          | 0/487 [00:00<?, ?ex/s]

  0%|          | 0/325 [00:00<?, ?ex/s]

In [ ]:
vocab = build_vocab_from_iterator(build_vocab(datasets.concatenate_datasets([train_dataset, valid_dataset, test_dataset]))['tokens'], specials=["<UNK>", "<EOS>"], min_freq=3)
vocab.set_default_index(vocab["<UNK>"])

  0%|          | 0/3245 [00:00<?, ?ex/s]

In [ ]:
#@title solo para ver que no nos hemos atrasado con el modelo
print(vocab.get_itos()[:10])


['<UNK>', '<EOS>', 'http', 'nuevo', 'león', 'vamos', 'hoy', 'gobierno', 'toda', 'política']


In [ ]:
batch_size = 32
train_data = batchify(tokenized_train_dataset, vocab, batch_size)
valid_data = batchify(tokenized_valid_dataset, vocab, batch_size)
test_data = batchify(tokenized_test_dataset, vocab, batch_size)

In [ ]:
vocab_size = len(vocab)
embedding_dim = 500
hidden_dim = 1200
num_layers = 2
dropout_rate = 0.5
lr = 1e-3
n_epochs = 100
seq_len = 40
clip = 0.25

In [ ]:
model = LSTM_LM(vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {num_params:,} trainable parameters')

The model has 24,689,934 trainable parameters


In [ ]:
saved = True

#reduce the learning rate by a factor of 2 after every epoch associated with no improvement
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=0)

In [ ]:
if saved:
    model.load_state_dict(torch.load('/content/drive/MyDrive/Tarea4_TSCD/tweets_politicos/best-val-SamuelGarcia.pt',  map_location=device))
    test_loss = evaluate(model, test_data, criterion, batch_size, seq_len, device)
    print(f'Test Perplexity: {math.exp(test_loss):.3f}')
else:
    best_valid_loss = float('inf')

    for epoch in range(n_epochs):
        train_loss = train(model, train_data, optimizer, criterion,
                    batch_size, seq_len, clip, device)
        valid_loss = evaluate(model, valid_data, criterion, batch_size,
                    seq_len, device)

        lr_scheduler.step(valid_loss)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), '/content/drive/MyDrive/Tarea4_TSCD/tweets_politicos/best-val-SamuelGarcia.pt')
        print(f'Epoch: {epoch}')
        print(f'\tTrain Perplexity: {math.exp(train_loss):.3f}')
        print(f'\tValid Perplexity: {math.exp(valid_loss):.3f}')

Test Perplexity: 251.995


Train Perplexity: 116.348

Valid Perplexity: 249.795

###Resultados

En este caso, solo implemente 3 palabras por que quiero ajustar el modelo destilado, pero en general al probar con diferentes palabras note que ene fecto, tiene a leerse bastante similar a su estilo, algo un pooc mas importante es que ante entradas como  "fueza civil" hablo justamente de palabras sumanente relacionadas al tema.

In [ ]:
#@title generate(agua)
prompt = 'agua'
generation = generate(prompt, max_seq_len, model, tokenizer, vocab, device)
print('Generated text:\n'+' '.join(generation)+'\n')

Generated text:
agua permite entras http gracias http adrián puro nuevo



In [ ]:
#@title generate_destilada(agua)
prompt = 'agua'
generation = generate_destilada(prompt, max_seq_len, model, tokenizer, vocab, device)
print('Generated text:\n'+' '.join(generation)+'\n')

Generated text:
agua rioja implementado garcía caigan trámites se_mx protocolos raniere



In [ ]:
prompt = 'Mariana'
generation = generate(prompt, max_seq_len, model, tokenizer, vocab, device)
print('Generated text:\n'+' '.join(generation)+'\n')

Generated text:
mariana sigue experto pronto vamos cuidar naranja http cgc6epr5ze



In [ ]:
prompt = 'fuerza civil'
generation = generate(prompt, max_seq_len, model, tokenizer, vocab, device)
print('Generated text:\n'+' '.join(generation)+'\n')

Generated text:
fuerza civil primeroloprimero mejores alcalde cómo nuevo león tipo puedan



## Marcelo Ebrard [Twitter](https://twitter.com/m_ebrard?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Eauthor)

###Construcción del modelo de lenguaje

In [ ]:
train_dataset, valid_dataset, test_dataset = leer_twets(MarceloEbrard)
tokenized_train_dataset, tokenized_valid_dataset, tokenized_test_dataset = TokenizerDatasets(train_dataset,valid_dataset,test_dataset)

  0%|          | 0/2428 [00:00<?, ?ex/s]

  0%|          | 0/486 [00:00<?, ?ex/s]

  0%|          | 0/324 [00:00<?, ?ex/s]

In [ ]:
vocab = build_vocab_from_iterator(build_vocab(datasets.concatenate_datasets([train_dataset, valid_dataset, test_dataset]))['tokens'], specials=["<UNK>", "<EOS>"], min_freq=3)
vocab.set_default_index(vocab["<UNK>"])

  0%|          | 0/3238 [00:00<?, ?ex/s]

In [ ]:
#@title solo para ver que no nos hemos atrasado con el modelo
print(vocab.get_itos()[:10])


['<UNK>', '<EOS>', 'http', 'méxico', 'sre_mx', 'presidente', 'hoy', 'gracias', 'obrador', 'lópez']


In [ ]:
batch_size = 32
train_data = batchify(tokenized_train_dataset, vocab, batch_size)
valid_data = batchify(tokenized_valid_dataset, vocab, batch_size)
test_data = batchify(tokenized_test_dataset, vocab, batch_size)

In [ ]:
vocab_size = len(vocab)
embedding_dim = 500
hidden_dim = 1200
num_layers = 2
dropout_rate = 0.5
lr = 1e-3
n_epochs = 100
seq_len = 40
clip = 0.25

In [ ]:
model = LSTM_LM(vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {num_params:,} trainable parameters')

The model has 24,021,441 trainable parameters


In [ ]:
saved = True

#reduce the learning rate by a factor of 2 after every epoch associated with no improvement
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=0)

In [ ]:
if saved:
    model.load_state_dict(torch.load('/content/drive/MyDrive/Tarea4_TSCD/tweets_politicos/best-val-MarceloEbrard.pt',  map_location=device))
    test_loss = evaluate(model, test_data, criterion, batch_size, seq_len, device)
    print(f'Test Perplexity: {math.exp(test_loss):.3f}')
else:
    best_valid_loss = float('inf')

    for epoch in range(n_epochs):
        train_loss = train(model, train_data, optimizer, criterion,
                    batch_size, seq_len, clip, device)
        valid_loss = evaluate(model, valid_data, criterion, batch_size,
                    seq_len, device)

        lr_scheduler.step(valid_loss)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), '/content/drive/MyDrive/Tarea4_TSCD/tweets_politicos/best-val-MarceloEbrard.pt')
        print(f'Epoch: {epoch}')
        print(f'\tTrain Perplexity: {math.exp(train_loss):.3f}')
        print(f'\tValid Perplexity: {math.exp(valid_loss):.3f}')

Test Perplexity: 179.937


Train Perplexity: 99.904

Valid Perplexity: 219.444

###Resultados

En este caso, su modelo de lenguaje es diferente, por ejemplo al del presidente, ya que mientras uno menciona ciudades en México, Ebrard habla de ciudades en el Extranjero, lo cual es consistsnte con su puesto como secretario de relaciones exteriores. al aplicar la función generate_destilada() elimine ese problema (aunque para sennecias muy largas se mantiene)

In [ ]:
#@title generate_destilada(cumbre)
prompt = 'cumbre'
generation = generate_destilada(prompt, max_seq_len, model, tokenizer, vocab, device)
print('Generated text:\n'+' '.join(generation)+'\n')

Generated text:
cumbre presidentes jmgomezrobledo alimentos ilícito aprueba acompañado acompañarnos haciendo



In [ ]:
#@title generate(naciones)
prompt = 'naciones'
generation = generate(prompt, max_seq_len, model, tokenizer, vocab, device)
print('Generated text:\n'+' '.join(generation)+'\n')

Generated text:
naciones tráfico harris gobierno pueblo http presidente crisis federación



In [ ]:
#@title generate_destilada(canciller)
prompt = 'canciller '
generation = generate_destilada(prompt, max_seq_len, model, tokenizer, vocab, device)
print('Generated text:\n'+' '.join(generation)+'\n')

Generated text:
canciller ucrania and people elena cuarta baptiste aduanas punto



In [ ]:
#@title generate_destilada(ucrania)
prompt = 'ucrania '
generation = generate_destilada(prompt, max_seq_len, model, tokenizer, vocab, device)
print('Generated text:\n'+' '.join(generation)+'\n')

Generated text:
ucrania telefónica acelerar mismos bronce esteban politica papa vehículos



#Conclusiones Generales:
En los 5 politicos, fui capaz de generar lenguaje que si bien no es perfecto, al mencionar las oraciones que tienen sentido a una persona externa, pudo detectar de que politico hablaba. El caso que tuvo mejores resultados fue el de Claudia, mientras que el peor fue el de Samuel